In [ ]:
import pandas as pd
df = pd.read_csv("report.csv")

In [4]:
sdf = df.sort_values('rt', ascending=False)

In [9]:
pd.set_option('display.max_colwidth', -1)
hdf = sdf[['date','request','rt']].head(100)

In [11]:
import urllib
hdf['request'] = hdf['request'].map(urllib.parse.unquote)
hdf

,date,request,rt
1803647,04/Sep/2020:11:05:01 +0000,"GET /fr/search/travelq?f[0]=:Commissariats à l’information et à la protection de la vie privée au Canada&f[1]=travelq_total_fr:Adjointe spéciale, opérations et planification&f[2]=travelq_total_fr:CSPPPM&f[3]=travelq_total_fr:Directeur exécutif et avocat général, Secrétariat du Tribunal canadien du commerce extérieur&f[4]=travelq_total_fr:Sous-ministre adjoint Direction générale de la gestion intégrée&f[5]=travelq_total_fr:Sous-ministre adjointe intérimaire, Secteur de la gestion des affaires publiques et du portefeuille&f[6]=travelq_total_fr:Vice-président, Affaires extérieures et développement d'entreprise HTTP/1.1",76.953
2771972,05/Sep/2020:15:33:21 +0000,"GET /fr/search/travelq?f[0]=:Conseil de recherches en sciences naturelles et en génie du Canada&f[1]=travelq_total_fr:Assistante à la Conseillère principale, Gestion des enjeux&f[2]=travelq_total_fr:Ministre du Développement économique et des Langues officielles&f[3]=travelq_total_fr:Sous-ministre adjoint, Politique stratégique, planification et affaires ministérielles&f[4]=travelq_total_fr:Sous-ministre adjoint principal, Services ministériels et dirigeant principal des finances &f[5]=travelq_total_fr:VP, Services corporatifs et Chef de la direction financière&f[6]=travelq_total_fr:Vice-présidente, Direction générale des affaires autochtones et du patrimoine culturel&f[7]=travelq_total_fr:Vice-présidente par intérim, Nouveau-Brunswick HTTP/1.1",75.058
2158831,07/Sep/2020:14:40:56 +0000,"GET /fr/search/travelq?f[0]=:Tribunal des anciens combattants (révision et appel)&f[1]=travelq_total_fr:Adjoint spécial, Région de l'Atlantique&f[2]=travelq_total_fr:Secrétaire parlemantaire&f[3]=travelq_total_fr:Sous-commissaire Principal, Administration Centrale&f[4]=travelq_total_fr:Vice-présidente, conseillère spéciale&f[5]=travelq_total_fr:Vice-présidente adjointe, SPIIE HTTP/1.1",71.040
1789164,04/Sep/2020:07:33:18 +0000,"GET /fr/search/travelq?f[0]=:Conseil de recherches en sciences naturelles et en génie du Canada&f[1]=travelq_total_fr:Ministre du Développement économique et des Langues officielles&f[2]=travelq_total_fr:Présidente-didrectrice générale&f[3]=travelq_total_fr:Secrétaire parlementaire pour l'Infrastructure et les Collectivités&f[4]=travelq_total_fr:Sous-ministre adjoint délégué, Direction des finances et échanges internationaux&f[5]=travelq_total_fr:Sous-ministre ajointe, Direction générale des programmes &f[6]=travelq_total_fr:VP, Services corporatifs et Chef de la direction financière&f[7]=travelq_total_fr:Vice-présidente par intérim, Nouveau-Brunswick HTTP/1.1",70.623
709777,09/Sep/2020:10:30:50 +0000,"GET /fr/search/travelq?f[0]=:Tribunal des anciens combattants (révision et appel)&f[1]=travelq_total_fr:Administrateur en chef des opérations&f[2]=travelq_total_fr:Ministre de l’Innovation, des Sciences et de l'Industrie&f[3]=travelq_total_fr:SPSC pour le Personnel Executif&f[4]=travelq_total_fr:Vice-présidente, Radiodiffusion &f[5]=travelq_total_fr:Vice président et Dirigeant principal HTTP/1.1",69.960
3094264,08/Sep/2020:15:50:10 +0000,"GET /fr/search/travelq?f[0]=:Commission civile d'examen et de traitement des plaintes relatives à la Gendarmerie royale du Canada&f[1]=travelq_total_fr:Chef - Réserves et appui de l'employeur (CRAE)&f[2]=travelq_total_fr:Conseiller politique principal&f[3]=travelq_total_fr:Directeur des politiques et affaires parlementaires&f[4]=travelq_total_fr:Sous-ministre adjoint, Direction générales des services à l'industrie et aux marchés&f[5]=travelq_total_fr:Vice président HTTP/1.1",69.785
709778,09/Sep/2020:10:30:50 +0000,"GET /fr/search/travelq?f[0]=:École de la fonction publique du Canada&f[1]=travelq_total_fr:Adjointe spéciale - Affaires parlementaires&f[2]=travelq_total_fr:Sous-ministre adjointe, Secteur des terres et des minéraux HTTP/1.1",69.156
1284643,07/Sep/2020:06:24:48 +0000,"GET /fr/search/travelq?f[0]=:Commissariats à l’information et à la protection de la vie privée au Cana

In [14]:
endf = sdf[sdf['request'].str.match('GET /en/search/travelq')][['date', 'request', 'rt']].head(100)
endf['request'] = endf['request'].map(urllib.parse.unquote)
endf

,date,request,rt
147521,04/Sep/2020:10:30:38 +0000,"GET /en/search/travelq?travelq[0]=org_name_en:Canada Revenue Agency&_ga=2.21213624.289352888.1528729136-1381001506.1471292866&f[0]=travelq_organization_en:Natural Sciences and Engineering Research Council of Canada&f[1]=travelq_organization_en:Office of the Auditor General of Canada&f[2]=travelq_organization_en:Veterans Review and Appeal Board&f[3]=travelq_total_en:B: $5,000.00 - $24,999.99&f[4]=travelq_total_en:F: $0.00 - $249.99 HTTP/1.1",43.302
186829,04/Sep/2020:18:06:01 +0000,GET /en/search/travelq?f[0]=:Statistics Canada&f[1]=travelq_organization_en:Canadian Centre for Occupational Health and Safety&f[2]=travelq_organization_en:Shared Services Canada&f[3]=travelq_year_en:2006&f[4]=travelq_year_en:2007&f[5]=travelq_year_en:2020&search_api_fulltext=&sort_by=date_clean&sort_order=DESC&f[0]=:Statistics Canada&f[1]=travelq_organization_en:Canadian Centre for Occupational Health and Safety&f[2]=travelq_organization_en:Shared Services Canada&f[3]=travelq_year_en:2006&f[4]=travelq_year_en:2007&f[5]=travelq_year_en:2020&page=1 HTTP/1.1,34.921
371502,06/Sep/2020:10:30:33 +0000,GET /en/search/travelq?f[0]=:Civilian Review and Complaints Commission for the RCMP&f[1]=travelq_organization_en:Correctional Service of Canada&f[2]=travelq_organization_en:Office of the Superintendent of Financial Institutions Canada&f[3]=travelq_organization_en:Secretariat of the National Security and Intelligence Committee of Parliamentarians&f[4]=travelq_organization_en:Status of Women Canada&f[5]=travelq_organization_en:Transport Canada&f[6]=travelq_total_en:E: $250.00 - $499.99&f[7]=travelq_year_en:2016 HTTP/1.1,32.956
147440,04/Sep/2020:10:28:58 +0000,GET /en/search/travelq?f[0]=:Statistics Canada&f[1]=travelq_organization_en:Office of the Commissioner for Federal Judicial Affairs Canada&f[2]=travelq_organization_en:Secretariat of the National Security and Intelligence Committee of Parliamentarians&f[3]=travelq_year_en:2003&f[4]=travelq_year_en:2006&f[5]=travelq_year_en:2008&f[6]=travelq_year_en:2020 HTTP/1.1,26.710
147556,04/Sep/2020:10:30:43 +0000,GET /en/search/travelq?travelq[0]=org_name_en:Department of Finance Canada&f[0]=travelq_organization_en:Canada Border Services Agency&f[1]=travelq_organization_en:Crown-Indigenous Relations and Northern Affairs Canada&f[2]=travelq_organization_en:Financial Transactions and Reports Analysis Centre of Canada&f[3]=travelq_organization_en:Military Grievances External Review Committee&f[4]=travelq_organization_en:Office of the Intelligence Commissioner HTTP/1.1,25.449
1307264,07/Sep/2020:10:04:56 +0000,GET /en/search/travelq?f[0]=:Atlantic Canada Opportunities Agency&f[1]=travelq_year_en:2018&search_api_fulltext=&sort_by=date_clean&sort_order=DESC HTTP/1.1,24.557
147580,04/Sep/2020:10:31:23 +0000,GET /en/search/travelq?travelq[0]=org_name_en:Department of Finance Canada&f[0]=travelq_organization_en:Canada Border Services Agency&f[1]=travelq_organization_en:Crown-Indigenous Relations and Northern Affairs Canada&f[2]=travelq_organization_en:Financial Transactions and Reports Analysis Centre of Canada&f[3]=travelq_organization_en:Office of the Intelligence Commissioner&f[4]=travelq_organization_en:The National Battlefields Commission HTTP/1.1,23.538
619022,08/Sep/2020:16:32:08 +0000,GET /en/search/travelq?f[0]=org_name_en:Shared Services Canada HTTP/1.1,22.372
147581,04/Sep/2020:10:31:23 +0000,GET /en/search/travelq?travelq[0]=org_name_en:Canada Revenue Agency&_ga=2.21213624.289352888.1528729136-1381001506.1471292866&f[0]=travelq_organization_en:Military Police Complaints Commission of Canada&f[1]=travelq_organization_en:Office of the Superintendent of Financial Institutions Canada&f[2]=travelq_organization_en:Office of the Taxpayers' Ombudsman&f[3]=travelq_year_en:2016&f[4]=travelq_year_en:2020 HTTP/1.1,22.081
325135,05/Sep/2020:23:17:41 +0000,GET /en/search/travelq?f[0]=:Civilian Review and Complaints Commission for the RCMP&f[1]=travelq_organization_en:Office 